In [1]:
import os
import sys
import xarray as xr
import numpy as np
import matplotlib
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from pathlib import Path
import cartopy.crs as ccrs
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
mcs_dir = Path('/neelin2020/mcs_flextrkr/mcs_stats/mcs_tracks_non2mcs')

In [4]:
# merge all tracks into a single xarray.dataset file 
ds_MCS = xr.open_dataset(mcs_dir / 'mcs_tracks_non2mcs_2014.tropics30NS.nc')

In [32]:
ds_MCSphase = []
track_list = []
for i in range(len(ds_MCS.tracks)):
    tmp = ds_MCS.isel(tracks=i)
    idt_phase = [0,
                 tmp.idt_mcs_init.values,
                 tmp.idt_mcs_grow.values,
                 tmp.idt_mcs_mature.values,
                 tmp.idt_mcs_decay.values,
                 tmp.idt_mcs_end.values]    
    tmp_phase = tmp.isel(times = idt_phase)
    tmp_phase['times'] = np.arange(6)
    tmp_phase = tmp_phase.rename({'times':'time'})
    
    track_list.append(tmp.tracks.values)
    ds_MCSphase.append(tmp_phase[['base_time','meanlat','meanlon']])  
ds_MCSphase_xr = xr.concat(ds_MCSphase, pd.Index(track_list, name='tracks'))

In [34]:
meanlon = ds_MCSphase_xr['meanlon']
meanlon_re = meanlon.where(meanlon >= 0, meanlon + 360)
ds_MCSphase_xr['meanlon'] = meanlon_re

In [35]:
out_dir = Path('/scratch/wmtsai/temp_mcs/mcs_stats/mcs_tracks_non2mcs/featenv_track_input')
ds_MCSphase_xr.to_netcdf(out_dir / 'MCS_FLEXTRKR_tropics30NS.2014.nc'
                     , encoding={'meanlon': {'dtype': 'float32'}, 'meanlat': {'dtype': 'float32'}})

In [38]:
track_info = ds_MCSphase_xr.isel(tracks=0)
idx_end = np.where(np.isnat(track_info.base_time.values))[0][0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [43]:
np.where(np.isnat(track_info.base_time.values))[0]

array([], dtype=int64)

In [44]:
track_info.base_time

<xarray.DataArray 'base_time' (time: 6)> Size: 48B
array(['2014-01-01T00:30:00.000013312', '2014-01-01T10:30:00.000000000',
       '2014-01-01T10:30:00.000000000', '2014-01-01T11:30:00.000026880',
       '2014-01-01T14:30:00.000026880', '2014-01-01T18:30:00.000013312'],
      dtype='datetime64[ns]')
Coordinates:
    tracks   int64 8B 30
  * time     (time) int64 48B 0 1 2 3 4 5
Attributes:
    long_name:  Epoch time of a feature